import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import matplotlib.ticker as mticker
import cartopy.crs as ccrs
import cartopy.feature as cfeature
dataset = xr.open_dataset('200301_202006-C3S-L3_GHG-PRODUCTS-OBS4MIPS-MERGED-v4.3.nc',engine='netcdf4')
time = dataset.variables['time']
lat = dataset.variables['lat'][:]
lon = dataset.variables['lon'][:]
xch4 = dataset.variables['xch4'][:]
months = 12
time = pd.to_datetime(time)
time_MONTH = time.month
climatology = np.zeros((months, xch4.shape[1], xch4.shape[2]))
for i in range(months):
    month_mask = (time_MONTH == i + 1) 
    month_data = xch4[month_mask]
    climatology[i] = np.mean(month_data, axis=0)
fig, axes = plt.subplots(4, 3, figsize=(15, 10))  
for i in range(months):
    ax = axes[i//3, i%3]
    im = ax.pcolormesh(lon, lat, climatology[i])
    ax.set_title(f'Month {i+1}')
    fig.colorbar(im, ax=ax)

plt.tight_layout()
plt.show()

##########1.2
#这里貌似不同版本的xarray会画出不同的图像
#基于xarray2023.6
global_mean = np.mean(xch4, axis=(1, 2)) 
plt.figure(figsize=(10, 5))
plt.plot(time, global_mean, label='Global Methane')
plt.title('Global Average Methane Levels (2003-2020)')
plt.xlabel('Time')
plt.ylabel('Methane (ppb)')
plt.legend()
plt.show()
#大气中的甲烷含量逐年增长，在每年的6、12月份左右，有低值

##########1.3
point_data = dataset.xch4.sel(lat=-15, lon=-150, method="nearest")
monthly_mean = point_data.groupby('time.month').mean('time')
deseasonalized_data = point_data.groupby('time.month') - monthly_mean
deseasonalized_data.plot()
plt.title('Deseasonalized Methane Levels at 15°S, 150°W (2003-01 to 2020-06)')
plt.xlabel('Time')
plt.ylabel('Methane Concentration (ppb)')
plt.show()

#############2
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import matplotlib.ticker as mticker
import cartopy.crs as ccrs
import cartopy.feature as cfeature
ds = xr.open_dataset('NOAA_NCDC_ERSST_v3b_SST.nc')
sst = ds['sst'].sel(lat=slice(-5, 5), lon=slice(120, 170))
monthly_climatology = sst.groupby('time.month').mean('time')
anomalies = sst.groupby('time.month') - monthly_climatology
#######2.2
nino34_index = anomalies.rolling(time=3, center=True).mean('time')
plt.figure(figsize=(10, 5))
nino34_index.mean(dim=['lat', 'lon']).plot()
plt.title('Niño 3.4 Index')
plt.xlabel('Year')
plt.ylabel('SST Anomaly (°C)')
plt.grid(True)
plt.show()

############3
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import matplotlib.ticker as mticker
import cartopy.crs as ccrs
import cartopy.feature as cfeature
#3.1
#因为助教发的数据好像每个Nc文件只有一个月的数据，所以我用了第二题的数据做3.1
ds = xr.open_dataset('NOAA_NCDC_ERSST_v3b_SST.nc')
print(ds)
sst = ds['sst']
monthly_climatology = sst.groupby('time.month').mean('time')
anomalies = sst.groupby('time.month') - monthly_climatology
plt.figure(figsize=(10, 5))
anomalies.mean(dim=['lat', 'lon']).plot()
plt.title('Time series')
plt.xlabel('Year')
plt.ylabel('SST Anomaly (°C)')
plt.grid(True)
plt.show()

#3.2
#读取了助教发的一个文件，绘制经纬度平均二氧化碳浓度图
ds = xr.open_dataset('oco2_GEOS_L3CO2_month_202202_B10206Ar.nc4')
print(ds)
#按经度
variable = ds['XCO2']
lon_mean = variable.mean(dim=['lat'])
plt.figure(figsize=(12, 6))
lon_mean.plot()
plt.title('Time Series of Mean XCO2')
plt.xlabel('lon')
plt.ylabel('XCO2 (ppm)')
plt.grid(True)
plt.show()
#按纬度
lat_mean = variable.mean(dim=['lon'])
plt.figure(figsize=(12, 6))
lat_mean.plot()
plt.title('Time Series of Mean XCO2')
plt.xlabel('lat')
plt.ylabel('XCO2 (ppm)')
plt.grid(True)
plt.show()
#绘制全球该时间的二氧化碳浓度，不知道为什么会有两个图例
plt.figure(figsize=(12, 8))
xco2_at_time = variable.isel(time=0)
fig, ax = plt.subplots()
mappable = xco2_at_time.plot(ax=ax, label='XCO2')  
plt.colorbar(mappable, label='XCO2 (ppm)', ax=ax)
plt.legend() 
plt.show()
########再读取一月的数据，奇怪的是上面读取二月分数据时直接输入文件名就行，这里输入文件名会报错
ds2 = xr.open_dataset('D:\py\output_file\oco2_GEOS_L3CO2_month_202201_B10206Ar.nc4')
#######绘制一月的全球浓度图
variable2 = ds2['XCO2']
plt.figure(figsize=(12, 8))
xco2_at_time2 = variable2.isel(time=0)
fig, ax = plt.subplots()
mappable = xco2_at_time2.plot(ax=ax, label='XCO2')  
plt.colorbar(mappable, label='XCO2 (ppm)', ax=ax)
plt.legend() 
plt.show()
#绘制二月相较于一月全球浓度变化图
#查看两者的形状
print(variable.sizes)
print(variable2.sizes)
print(variable.dims)
print(variable2.dims)
#两者形状明明相同但是ValueError: operands cannot be broadcast together with mismatched lengths for dimension 'lat': (361, 1)
#把时间去掉后进行做差
variable_squeezed = variable.squeeze('time')
variable2_squeezed = variable2.squeeze('time')
var=variable_squeezed-variable2_squeezed
plt.figure(figsize=(10, 6))
fig, ax = plt.subplots()
mappable = var.plot(ax=ax)
plt.colorbar(mappable, ax=ax)
plt.show()